# Intruduction to NLP fundamentals in TensorFlow
NLP has the goal of deriving information out of natural language (could be sequences of text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

## Check for the GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-fb29eec0-b22f-99f2-952c-d908a5368a5f)


## Get helper functions

In [2]:
# Download the helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-03-20 20:09:30--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-03-20 20:09:30 (83.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset
The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labeled as disaster or not disaster).

See the orignal source here: https://www.kaggle.com/competitions/nlp-getting-started

In [3]:
# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data('nlp_getting_started.zip')

--2023-03-20 20:09:38--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.107.128, 173.194.202.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.107.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.008s  

2023-03-20 20:09:38 (68.7 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Become one with the data - visualize a text dataset

To visualize our text sample, we first need to read them in. One way to do so would be to use Python.

But we can get visual straight away.

So another way to do this is to use Pandas.

TF help: https://www.tensorflow.org/tutorials/load_data/text

In [4]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [6]:
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, 
                                    random_state=42)

train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
# What does the test dataset look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples are there?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

If dataset is imbalanced: https://www.tensorflow.org/tutorials/structured_data/imbalanced_data

In [10]:
# How many total samples
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Let's visualize some random training examples
import random 
random_index = random.randint(0, len(train_df)-5) #create random indexes not higher then random number of samples

for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text: \n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text: 
Nazi Mass Murderer Became Chairman At Vaccine Drug Company In... http://t.co/x713OMh6Ai

---

Target: 0 (not real disaster)
Text: 
Diageo's CEO stresses that a board revolt at United Spirits has not impacted Indian operations http://t.co/STPOdA901U

---

Target: 0 (not real disaster)
Text: 
Blending the old with the new in #Vanuatu to prepare for future emergencies: 
http://t.co/aFMKcFn1TL http://t.co/8QqzYZIAqf

---

Target: 0 (not real disaster)
Text: 
Excited for Cyclone football https://t.co/Xqv6gzZMmN

---

Target: 1 (real disaster)
Text: 
Malaysian Officials Say Debris Found on Reunion Island Is From #MH370. @BillNeelyNBC reports: http://t.co/r6kZSQDghZ

---



### Split data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# Use train_test_split to split the training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'],
                                                                            train_df_shuffled['target'],
                                                                            test_size=0.1,
                                                                            random_state=42)

In [14]:
# Check the lenghts
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [15]:
# Check some top samples
train_sentences[:10], train_labels[:10]

(5921    @mogacola @zamtriossu i screamed after hitting...
 3904              Imagine getting flattened by Kurt Zouma
 2804    @Gurmeetramrahim #MSGDoing111WelfareWorks Gree...
 3718    @shakjn @C7 @Magnums im shaking in fear he's g...
 1667    Somehow find you and I collide http://t.co/Ee8...
 4435    @EvaHanderek @MarleyKnysh great times until th...
 2544                     destroy the free fandom honestly
 7223    Weapons stolen from National Guard Armory in N...
 4265    @wfaaweather Pete when will the heat wave pass...
 6568    Patient-reported outcomes in long-term survivo...
 Name: text, dtype: object, 5921    0
 3904    0
 2804    1
 3718    0
 1667    0
 4435    1
 2544    1
 7223    0
 4265    1
 6568    1
 Name: target, dtype: int64)

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text into numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character) to number
* Embeddings - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embeddings can be learned)

### Text vectorizaton - tokenization

In [16]:
train_sentences[:5]

5921    @mogacola @zamtriossu i screamed after hitting...
3904              Imagine getting flattened by Kurt Zouma
2804    @Gurmeetramrahim #MSGDoing111WelfareWorks Gree...
3718    @shakjn @C7 @Magnums im shaking in fear he's g...
1667    Somehow find you and I collide http://t.co/Ee8...
Name: text, dtype: object

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [18]:
# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automaticall add <OOV>)
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, # create groups of n-words?
                                    output_mode='int', # how to map tokens to numbers
                                    output_sequence_length=None, # how long you want your sequences to be
                                    #pad_to_max_tokens=True
                                    )

In [19]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [20]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words in vocabulary
max_length = 15 # max length our sequences will be (e.g. for many words will a model see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [21]:
# Fit the text vextorizer to the training set
text_vectorizer.adapt(train_sentences)

In [22]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [24]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\nVectorized version: ")
text_vectorizer([random_sentence])

Original text:
 #YIAYplan Use my awesome collection of Amiibos to destroy all in my path.      

Vectorized version: 


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[6559,  447,   13, 1042, 2517,    6,    1,    5,  305,   44,    4,
          13, 2809,    0,    0]])>

In [25]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in training data
top_5_words = words_in_vocab[:5] # most 5 common words
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words are: {top_5_words}")
print(f"Top 5 least common words are: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words are: ['', '[UNK]', 'the', 'a', 'in']
Top 5 least common words are: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Create and Embedding using and Embedding Layer
To make our embedding we're going to use TensorFlow's Embedding Layer.

Parameters we care the most are:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example: a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of sequences being passes to the embedding layer

In [26]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # output shape
                             input_length=max_length #how long in each input
                             )

embedding

In [27]:
# Get another random sentence
random_sentence = random.choice(train_sentences)
print(f"Original text: {random_sentence}")
print(f"Embedded version: ")

# Embed the random sentence (turn it into the dense vector of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text: So Pardon me while I burst into flames. 

I've had enough of the world and its people's mindless games
Embedded version: 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00567127,  0.00506343,  0.01112581, ...,  0.00722972,
         -0.01302503, -0.00429493],
        [ 0.04649547, -0.01456211, -0.00571533, ..., -0.04408261,
         -0.04363103,  0.04320667],
        [-0.01302866,  0.02157117,  0.03105327, ..., -0.04391908,
          0.02024763,  0.0137335 ],
        ...,
        [ 0.01474135,  0.03139642,  0.02685589, ..., -0.00659534,
         -0.00869213,  0.04731229],
        [-0.04507516, -0.03113201,  0.02356524, ..., -0.03291929,
          0.00087693,  0.04316464],
        [-0.04126916, -0.00844779, -0.00641849, ..., -0.01213389,
          0.04732964,  0.02128706]]], dtype=float32)>

In [28]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.00567127,  0.00506343,  0.01112581, -0.01030596,  0.036139  ,
         0.02320304, -0.0403108 ,  0.04719828,  0.048188  ,  0.02666218,
        -0.00548253, -0.04212876, -0.00144693, -0.03029623, -0.02218919,
         0.01012801,  0.00083392, -0.02928736, -0.00466243,  0.02220038,
        -0.0039028 , -0.03051392,  0.03029693, -0.01843767, -0.04227519,
        -0.01004211,  0.03950066,  0.02666963, -0.00469742, -0.04263895,
        -0.03606453, -0.01756043,  0.04491786,  0.00997347,  0.03296998,
         0.00348532, -0.0206587 , -0.02982547, -0.04289619,  0.00685038,
        -0.00808066,  0.01849624,  0.02840978, -0.01891259,  0.02754072,
        -0.02586486, -0.01664323,  0.00136161,  0.02172628,  0.01923232,
         0.00197639, -0.03825412,  0.00755323, -0.01481931, -0.04833722,
         0.00807483, -0.04197709,  0.02686849,  0.01471634,  0.03704948,
         0.04927595,  0.02996624,  0.03299322, -0.0111299 ,  0.00290527,
  

## Modelling a text dataset (running a bunch of experiments)

Now we've got a way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline), this is from Sklearn ML Map
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM Model (RNN)
* Model 3: GRU Model (RNN)
* Model 4: Bidirectional-LSTM (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6, but with 10% training data

### Model 0: getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon. 

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

> **NOTE:** It's common practice to use non-DL algorythms as a baseline model because of their speed and then later using DL to see if you can improve upon them.


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()), # convert words to numbers using TF-IDF
    ('clf', MultinomialNB()) # model the text
])

# Fit the pipeline to the data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [30]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [31]:
# Make predictions 
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time. However this may bu cumbersom and will surely take some time.

Let's create one to compare our model's predcitions with truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1-score

In [32]:
# Function to evaluate: accuracy, recall, precision, f1-score

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculate model accuracy, recall, precision and f1-score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100

  # Calculate precision, recall and f1-score using 'weighted' metrics
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

  model_results = {'accuracy': model_accuracy,
                   'precision': model_precision,
                   'recall': model_recall,
                   'f1': model_f1}

  return model_results

In [33]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A simple dense model

In [34]:
# Create a tensorboard callback (need to create new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [35]:
# Build model with Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to each vector
outputs = layers.Dense(1, activation='sigmoid')(x) # create the output layer, want binary output so sigmoid

model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')

In [36]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [37]:
# Compile model
model_1.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [38]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name='model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20230320-201005
Epoch 1/5
215/215 [==============================] - 16s 51ms/step - loss: 0.6116 - accuracy: 0.6993 - val_loss: 0.5366 - val_accuracy: 0.7467
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4414 - accuracy: 0.8184 - val_loss: 0.4664 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3461 - accuracy: 0.8615 - val_loss: 0.4582 - val_accuracy: 0.7953
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2836 - accuracy: 0.8937 - val_loss: 0.4624 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2371 - accuracy: 0.9098 - val_loss: 0.4808 - val_accuracy: 0.7874


In [39]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 15ms/step - loss: 0.4808 - accuracy: 0.7874


[0.4807589054107666, 0.787401556968689]

In [40]:
# Make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


array([[0.29134595],
       [0.7973998 ],
       [0.99765956],
       [0.1154449 ],
       [0.10367263],
       [0.9297371 ],
       [0.91697145],
       [0.99402547],
       [0.96693325],
       [0.26452464]], dtype=float32)

In [41]:
# Single prediction
model_1_pred_probs[1]

array([0.7973998], dtype=float32)

In [42]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [43]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7937136229340627,
 'recall': 0.7874015748031497,
 'f1': 0.7839588199365206}

In [44]:
import numpy as np
np.array(list(model_1_results.values()))> np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualizing learned embeddings

In [45]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [46]:
# Model_1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [47]:
# Get the weight matrix of embedding layer
# (these are the numerical representation of each token in our training data
# which have been learned for 5 epochs)
embed_weights = model_1.get_layer('embedding').get_weights()[0]
embed_weights

array([[-0.00813535,  0.05309479, -0.03743252, ...,  0.00159676,
        -0.02570872,  0.0287853 ],
       [-0.02877286,  0.0068551 , -0.03750751, ..., -0.00783495,
         0.05365443, -0.02636851],
       [ 0.02150869,  0.0379645 ,  0.02015401, ..., -0.00020607,
        -0.00110652,  0.05330781],
       ...,
       [ 0.03978196,  0.00194919,  0.00800475, ...,  0.02893524,
        -0.03293367, -0.01624108],
       [ 0.06176232,  0.05304624, -0.00912348, ..., -0.00778498,
         0.01627236,  0.00082677],
       [ 0.11364003,  0.09598613, -0.08639085, ...,  0.08596253,
         0.11156349,  0.03237244]], dtype=float32)

In [48]:
embed_weights.shape # same size as vocab size and embedding_dim (output_dim of our embedding layer)

(10000, 128)

Now we've got the embedding matrix, our model has learned to represent our tokens. Let's see how we can visualize it.

To do so, TensorFlow has a handy tool called projector: http://projector.tensorflow.org/

And TensorFlow also has an incredible guide on word embeddings themselves.

In [49]:
# Code straight from TensorFlow documentation
# Create embedding files
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [50]:
# Download files from Colab to import them to Projector
# Code straight from TensorFlow documentation
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks (RNN's)

RNN's are useful for sequential data.

The premise of a recurrent neural network is to use the representation of a precious input to aid the representation of a later input.

### Model 2: LSTM

LSTM = Long short term memory (one of the most popular RNN cells).

Our structure of an RNN typically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/Dense) -> Output (label probability)



In [51]:
# Create an LSTM model

from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
#print(x.shape)
#x = layers.LSTM(units=64, return_sequences=True)(x) # when you're stacking RNN cells together, you need to return_sequences=True
#print(x.shape)
x = layers.LSTM(64)(x)
#print(x.shape)
#x = layers.Dense(64, activation='relu')(x)
#print(x.shape)
outputs = layers.Dense(1, activation='sigmoid')(x)

model_2 = tf.keras.Model(inputs, outputs, name='model_2_LSTM')

In [52]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [53]:
# Compile the model
model_2.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [54]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     'model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230320-201052
Epoch 1/5
215/215 [==============================] - 25s 78ms/step - loss: 0.2223 - accuracy: 0.9241 - val_loss: 0.6144 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1560 - accuracy: 0.9418 - val_loss: 0.6134 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1289 - accuracy: 0.9523 - val_loss: 0.6861 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1026 - accuracy: 0.9603 - val_loss: 0.7456 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0804 - accuracy: 0.9670 - val_loss: 0.8943 - val_accuracy: 0.7625


In [55]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 6ms/step


array([[0.7608396 ],
       [0.88158923],
       [0.99988973],
       [0.17720632],
       [0.00234367],
       [0.99975485],
       [0.95888853],
       [0.9999343 ],
       [0.99989986],
       [0.5593563 ]], dtype=float32)

In [56]:
# Convert model_2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [57]:
# Calculate model_2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 76.24671916010499,
 'precision': 0.7623201541471376,
 'recall': 0.7624671916010499,
 'f1': 0.7623818109050775}

In [58]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 3: GRU
Another popular and effective RNN component is the GRU layer (or Gated Recurrent Unit).

The GRU cell has similar features to LSTM cell but has less parameters.

In [73]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.GRU(64, return_sequences=True)(x) # return_sequences=True if stacking layers
#x = layers.LSTM(64, return_sequences=True)(x)
#x = layers.GRU(64)(x)
#x = layers.Dense(64, activation='relu')(x)
#x = layers.GlobalAveragePooling1D()(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model_3 = tf.keras.Model(inputs, outputs, name='model_3_GRU')

In [74]:
# Check the summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_13 (GRU)                (None, 64)                37248     
                                                                 
 dense_12 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [75]:
# Compile the model
model_3.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [76]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20230320-203021
Epoch 1/5
215/215 [==============================] - 16s 57ms/step - loss: 0.1587 - accuracy: 0.9377 - val_loss: 0.7444 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0826 - accuracy: 0.9698 - val_loss: 0.8821 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0713 - accuracy: 0.9726 - val_loss: 1.0217 - val_accuracy: 0.7769
Epoch 4/5
215/215 [==============================] - 2s 12ms/step - loss: 0.0603 - accuracy: 0.9755 - val_loss: 1.1488 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0561 - accuracy: 0.9758 - val_loss: 1.0868 - val_accuracy: 0.7664


In [79]:
# Make predictions with GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 4ms/step


array([[2.2072815e-03],
       [7.3781770e-01],
       [9.9962544e-01],
       [2.4657530e-01],
       [1.5394372e-04],
       [9.9939311e-01],
       [7.6667035e-01],
       [9.9991405e-01],
       [9.9976450e-01],
       [8.6754119e-01]], dtype=float32)

In [80]:
# Convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [82]:
# Calculate model 3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 76.64041994750657,
 'precision': 0.7665895370389821,
 'recall': 0.7664041994750657,
 'f1': 0.7651213533864446}

### Model 4: Bidirectional RNN

Normal RNNs go from left to right (just the same way you read an English sentence). However, bidirectional RNN goes from right to left as well as from left to right.

In [86]:
# Create a bidirectional RNN model 4
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model_4 = tf.keras.Model(inputs, outputs, name='model_4_bidirectional')

In [87]:
# Check the summary
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_14 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [88]:
# Compile the model
model_4.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [89]:
# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_4_bidirectional')])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230320-205406
Epoch 1/5
215/215 [==============================] - 14s 47ms/step - loss: 0.0982 - accuracy: 0.9730 - val_loss: 0.9688 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0535 - accuracy: 0.9775 - val_loss: 1.2274 - val_accuracy: 0.7651
Epoch 3/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0437 - accuracy: 0.9794 - val_loss: 1.2140 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 4s 16ms/step - loss: 0.0414 - accuracy: 0.9804 - val_loss: 1.4433 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 4s 21ms/step - loss: 0.0381 - accuracy: 0.9807 - val_loss: 1.4523 - val_accuracy: 0.7743


In [91]:
# Make predicitons
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 5ms/step


array([[1.5271998e-04],
       [6.1373568e-01],
       [9.9991918e-01],
       [2.9990944e-01],
       [7.1652589e-06],
       [9.9886656e-01],
       [3.7200424e-01],
       [9.9996674e-01],
       [9.9991143e-01],
       [8.4643549e-01]], dtype=float32)

In [92]:
# Convert predictions
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [93]:
# Calculate results
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)
model_4_results

{'accuracy': 77.42782152230971,
 'precision': 0.7790421904139229,
 'recall': 0.7742782152230971,
 'f1': 0.77094690659836}

## Convolutional Neural Networks for Text (and other sequences)

We've used CNNs for images, but images are typically 1D (H x W), however our text data is 1D.

Previoulsy we've used `Conv2D()` for image data but now we're going to use `Conv1D()` as data is 1-dimensional.

The typical structure of a Conv1D model for sequences (in our case, text):
```
Inputs (text) -> Tokenization -> Embedding -> Layer(s) (typically Conv1D + pooling) -> Outputs
```

### Model 5: 1D Convolutional Neural Network

In [99]:
# Test out our embedding layer, Conv1D layer and MaxPooling
from tensorflow.keras import layers

embedding_test = embedding(text_vectorizer(['this is a test sequence'])) # turn target sequence into embedding
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5, # also referred as an ngram=5 (looks at 5 words at a time)
                        strides=1, # default
                        activation='relu',
                        padding='same') # default = valid, output is smaller than input, same -> input is same shape as output
conv_1d_output = conv_1d(embedding_test) # pass embedding through Conv1D layer
max_pool = layers.GlobalMaxPooling1D()
max_pool_output = max_pool(conv_1d_output) # equaivalent to 'get the most important feature/feature with the highest value'

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 32]), TensorShape([1, 32]))

In [103]:
#embedding_test

In [104]:
#conv_1d_output

In [105]:
#max_pool_output